Step 1: 
- Import df
- df = 'url', 'name', 'price', 'color', 'images', 'image link'

In [221]:
import pandas as pd

# CAREFUL WHEN REDUCING:
#       Need to delete faiss_index.bin 
# downloaded = 500
num_samples = 100

# Load dataset
df = pd.read_csv("./datasets/products.csv").head(num_samples)

# Keep only relevant columns
df = df[['url', 'name', 'price', 'color', 'images', 'image link']]

# Display first few rows
print(df.head())

                                                 url  \
0  https://www.asos.com/new-look/new-look-trench-...   
1  https://www.asos.com/stradivarius/stradivarius...   
2  https://www.asos.com/jdy/jdy-oversized-trench-...   
3  https://www.asos.com/nike-running/nike-running...   
4  https://www.asos.com/asos-curve/asos-design-cu...   

                                                name  price    color  \
0                      New Look trench coat in camel  49.99  Neutral   
1     Stradivarius double breasted wool coat in grey  59.99     GREY   
2                 JDY oversized trench coat in stone  45.00    STONE   
3                 Nike Running hooded jacket in pink  84.95     Pink   
4  ASOS DESIGN Tall linen mix trench coat in natural  75.00  Natural   

                                              images  \
0  ['https://images.asos-media.com/products/new-l...   
1  ['https://images.asos-media.com/products/strad...   
2  ['https://images.asos-media.com/products/jdy-o...   
3  ['h

Step 2: (takes time)
- Download image
- Extract description
- Generate description
- Apply LLaVa
- df = 'url', 'name', 'price', 'color', 'images', 'image link' + description, image path

In [222]:
import os
import subprocess
import requests
import shlex 
import re

# Define LLaVA paths (modify these based on your system)
LLAVA_CLI_PATH = "/Users/glennsuristio/Documents/Projects/dressAI/llama.cpp/llama-llava-cli"
LLAVA_MODEL_PATH = "/Users/glennsuristio/Documents/Projects/dressAI/llava-v1.6-mistral-7b/Mistral-7B-Instruct-v0.2-F32-Q4_K_M.gguf"
MM_PROJ_PATH = "/Users/glennsuristio/Documents/Projects/dressAI/llama.cpp/vit/mmproj-model-f16.gguf"
IMAGES_DATA = "./images_data"
DESCRIPTIONS_DATA = "./descriptions_data"

# Ensure temp directory exists
os.makedirs(IMAGES_DATA, exist_ok=True)
os.makedirs(DESCRIPTIONS_DATA, exist_ok=True)

def download_image(image_url, filename):
    """Downloads an image from a URL if it doesn't already exist."""
    if os.path.exists(filename):
        print(f"Image already exists: {filename}")
        return filename
    
    """Downloads an image from a URL and saves it locally."""
    response = requests.get(image_url, stream=True)
    if response.status_code == 200:
        with open(filename, 'wb') as file:
            for chunk in response.iter_content(1024):
                file.write(chunk)

        print(f"Downloaded: {filename}")
        return filename
    else:
        print(f"Failed to download image: {image_url}")
        return None

def extract_description(output_file):
    """Extracts only the relevant product description from LLaVA output."""
    with open(output_file, "r") as f:
        lines = f.readlines()

    # Find the starting point of the description
    start_index = None
    for i, line in enumerate(lines):
        if "encode_image_with_clip: image encoded" in line:
            start_index = i + 1  # Description starts on the next line
            break

    # Extract everything after the start_index
    if start_index is not None and start_index < len(lines):
        return " ".join(lines[start_index:]).strip()
    else:
        return "Description not found"

def sanitize_filename(name):
    """Replaces special characters that are invalid in filenames."""
    name = name.replace(' ', '_')  # Replace spaces with underscores
    return re.sub(r'[\\/:"*?<>|]', '_', name)  # Replace `/ \ : " * ? < > |` with `_`

def generate_description(image_url, name, color):
    """Generates a textual description using LLaVA for a given fashion product image."""
    safe_name = sanitize_filename(name)
    image_path = os.path.join(IMAGES_DATA, f"{safe_name}.jpg")
    description_path = os.path.join(DESCRIPTIONS_DATA, f"{safe_name}.txt")

    # If description file already exists, read from it
    if os.path.exists(description_path):
        with open(description_path, "r") as file:
            existing_description = file.read().strip()

        if existing_description and existing_description != "Description not found":
            print(f"✅ Using existing description for {name}")
            return existing_description
        else:
            print(f"🔄 Regenerating description for {name} (previously invalid)")

        # print(f"Reading existing description for {name}")
        # return open(description_path, "r").read()

    downloaded_image = download_image(image_url, image_path)
    if not downloaded_image:
        return "Image not available"
    
    output_file = "llava_output.txt"
    
#     prompt = f"""{image_path}
# USER:
# Describe the {color} {name} in this image in detail.
# - Focus on its fabric, style, and patterns.
# - Ignore other clothing items other than {color} {name} in the image.
# - Do NOT add any extra information other than the description.
# - Write the response as a single detailed paragraph. Do not use bullet points.
# - Avoid listing features separately; instead, describe the product naturally in a flowing sentence.

# ASSISTANT:
# """
    
    prompt = f"""
USER:
Describe the {color} {name} in this image in detail.
- Focus on its **fabric, style, patterns, and overall aesthetic**.
- Mention the **fit** (e.g., loose, tight, relaxed), **comfort level**, and **mobility**.
- Describe the **material and texture** (e.g., soft cotton, thick wool, waterproof fabric).
- Indicate whether it is **suitable for certain weather conditions** (e.g., breathable for summer, ideal for rainy days).
- Suggest occasions it is best suited for (e.g., casual, formal, date night, outdoor wear, business attire).
- Optionally mention what it might **pair well with** (e.g., jeans, sneakers, high heels, trench coat).
- Ignore other clothing items in the image and focus only on the {color} {name}.
- Do NOT add any extra information outside the description.
- Write the response as a **single paragraph** with **natural, flowing sentences**.

ASSISTANT:
"""


    command = f'{LLAVA_CLI_PATH} -m {LLAVA_MODEL_PATH} --mmproj {MM_PROJ_PATH} --image {shlex.quote(image_path)} -c 4096 -p "{prompt}" > {output_file}'
    print(f"Running Command: {command}")  # Debugging
    
    process = subprocess.run(command, shell=True, capture_output=True, text=True)
    
    print("STDOUT:", process.stdout)  # Debugging
    print("STDERR:", process.stderr)  # Debugging
    
    # Extract clean description
    description = extract_description(output_file)

    # Save the description to a file
    with open(description_path, "w") as desc_file:
        desc_file.write(description)
        
    print(f"Generated Description for {name}: {description}")
    return description

# Apply LLaVA on limited products
df['description'] = df.apply(lambda row: generate_description(row['image link'], row['name'], row['color']), axis=1)
df['image_path'] = df.apply(lambda row: os.path.join(IMAGES_DATA, f"{row['name'].replace(' ', '_')}.jpg"), axis=1)

print("Descriptions generated and saved!")


✅ Using existing description for New Look trench coat in camel
✅ Using existing description for Stradivarius double breasted wool coat in grey
✅ Using existing description for JDY oversized trench coat in stone
✅ Using existing description for Nike Running hooded jacket in pink
✅ Using existing description for ASOS DESIGN Tall linen mix trench coat in natural
✅ Using existing description for ASOS DESIGN denim bomber in ecru
✅ Using existing description for ASOS Weekend Collective nylon track jacket in neutral
✅ Using existing description for ASOS DESIGN Tall ultimate faux leather biker jacket in black
✅ Using existing description for Native Youth oversized twill shacket co-ord in purple
✅ Using existing description for Carhartt WIP michigan OG jacket in black
✅ Using existing description for ASOS DESIGN denim short sleeve shirt in midwash blue
✅ Using existing description for Bershka maxi belted coat in black
✅ Using existing description for Vero Moda trench coat in black
✅ Using exist

Step 3: (takes time)
- Generate embedding with MiniLM-L6-v2
- Convert embeddings into FAISS index

In [223]:
import os
import faiss
import numpy as np
import pandas as pd
from sentence_transformers import SentenceTransformer

# Directory for FAISS index
FAISS_DIR = "faiss_data"
os.makedirs(FAISS_DIR, exist_ok=True)

# File path for FAISS index
FAISS_INDEX_FILE = os.path.join(FAISS_DIR, "faiss_index.bin")

# Load embedding model
embedding_model = SentenceTransformer("paraphrase-MiniLM-L6-v2")

# Step 1: Load FAISS index if it exists
if os.path.exists(FAISS_INDEX_FILE):
    print("Loading existing FAISS index...")
    index = faiss.read_index(FAISS_INDEX_FILE)
    num_existing = index.ntotal  # Number of stored products in FAISS

    # Retrieve stored embeddings only if FAISS has indexed products
    if num_existing > 0:
        existing_embeddings = np.zeros((num_existing, index.d), dtype=np.float32)
        index.reconstruct_batch(np.arange(num_existing), existing_embeddings)  # Batch retrieval

        # Assign stored embeddings back to df
        df.loc[df.index[:num_existing], "embedding"] = pd.Series(list(existing_embeddings))

    # Identify new products that don't have an index yet
    new_products = df[df["embedding"].isna()]

else:
    print("Creating new FAISS index...")
    index = None  # Placeholder for FAISS index
    new_products = df  # All products are new

# Step 2: Generate embeddings ONLY for new products
if not new_products.empty:
    print(f"Found {len(new_products)} new products. Updating FAISS index...")

    # 🚀 **Batch Encode New Descriptions**
    new_embeddings = embedding_model.encode(
        new_products["description"].tolist(),
        batch_size=32,
        convert_to_numpy=True
    ).astype(np.float32)  # Ensure correct dtype

    new_embeddings = np.array(new_embeddings, dtype=np.float32)

    # Store embeddings in df using `.loc`
    df.loc[new_products.index, "embedding"] = pd.Series(list(new_embeddings), index=new_products.index)


    # Convert to FAISS format
    new_embeddings_array = np.vstack(df.loc[new_products.index, "embedding"].to_numpy())

    new_embeddings_array = np.array(new_embeddings_array, dtype=np.float32)

    # Add new embeddings to FAISS
    if index is None:
        index = faiss.IndexFlatL2(new_embeddings_array.shape[1])  # Create FAISS index
    index.add(new_embeddings_array)

    # Save updated FAISS index
    faiss.write_index(index, FAISS_INDEX_FILE)
    print("FAISS index updated with new products!")

else:
    print("No new products found. Using existing FAISS index.")

print("FAISS index ready & embeddings stored in df!")


Loading existing FAISS index...
No new products found. Using existing FAISS index.
FAISS index ready & embeddings stored in df!


Step 4:
- Retrieve top 3 products with cosine similarity

In [224]:
import numpy as np
import faiss
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

def retrieve_relevant_products(query, top_k=3):
    """
    Hybrid retrieval: Combines FAISS (dense) and TF-IDF (sparse) for better search.
    """
    #Convert query to FAISS embedding
    query_embedding = embedding_model.encode([query], convert_to_numpy=True).reshape(1, -1)

    #Retrieve from FAISS
    top_k_faiss = top_k * 3  # Retrieve more to rerank better
    distances, indices = index.search(query_embedding.astype(np.float32), top_k_faiss)
    retrieved_products = df.iloc[indices[0]].copy()  # Use .copy() to avoid modifying original df

    #Compute cosine similarity for reranking
    product_embeddings = np.vstack(retrieved_products["embedding"].to_numpy())
    similarity_scores = cosine_similarity(query_embedding, product_embeddings)[0]
    retrieved_products["similarity"] = similarity_scores

    #Sparse Retrieval Using TF-IDF
    vectorizer = TfidfVectorizer(stop_words="english")
    tfidf_matrix = vectorizer.fit_transform(df["name"] + " " + df["description"])  # Use both name & description
    query_vector = vectorizer.transform([query])

    #Compute TF-IDF similarity scores for retrieved products only
    sparse_scores = np.array((tfidf_matrix[retrieved_products.index] @ query_vector.T).todense()).flatten()

    #Assign sparse scores only to retrieved products
    retrieved_products["sparse_score"] = sparse_scores

    # Normalize scores and rerank
    retrieved_products["final_score"] = (
        0.7 * retrieved_products["similarity"] +  # Dense search weight
        0.3 * retrieved_products["sparse_score"]  # Sparse search weight
    )
    retrieved_products = retrieved_products.sort_values("final_score", ascending=False)

    return retrieved_products.head(top_k)


Step 5:
- Generate rag text response
- Pass in result details

In [225]:
# import gradio as gr
# import ollama 

# def generate_rag_response(user_query):
#     """
#     Uses Ollama to generate structured product recommendations with explanations, styling tips, and alternative suggestions.
#     """
#     retrieved_products = retrieve_relevant_products(user_query, top_k=3)

#     if retrieved_products.empty:
#         return []

#     #Create a formatted product list for AI to process
#     product_list = "\n\n".join(
#         [
#             f"*Name:* {row['name']}\n"
#             f"*Description:* {row['description']}\n"
#             for i, row in retrieved_products.iterrows()
#         ]
#     )

#     #Construct the AI prompt
#     prompt = f"""
# User Query: "{user_query}"

# The following fashion products were retrieved as the most relevant matches:

# {product_list}

# Act as a fashion AI assistant. For each product, explain why it was chosen, how it matches the user's request, and provide styling tips.
# Respond with a brief and short structured paragraph for each product.
# Don't display the name of the product in your response.
# Just give your explanations as instructed.
# """

#     #Generate response with Ollama
#     response = ollama.chat(
#         model="mistral", messages=[{"role": "user", "content": prompt}]
#     )
#     print(response)
#     response_text = response["message"]["content"]

#     #Split the AI response into separate product explanations
#     explanations = response_text.split("\n\n")  # Splitting paragraphs for individual products
    
#     #Format structured output for the chat
#     structured_recommendations = []
#     for (index, row), explanation in zip(retrieved_products.iterrows(), explanations):
#         structured_recommendations.append(
#             {
#                 "text": f"**{row['name']}**\n\n**${row['price']}**\n\n{explanation[3:]}\n\n🔗 [View Product]({row['url']})",
#                 "image": row["image link"],
#                 "images": row["images"]
#             }
#         )

#     return structured_recommendations  # Return structured list of text + images


In [226]:
# '''
# STEP 5
# '''

# import gradio as gr
# import ollama 

# def generate_rag_response(user_query, criteria=None, session=None):
#     """
#     Uses Ollama to generate structured product recommendations with explanations.
    
#     Args:
#         user_query (str): The user's search query
#         criteria (dict, optional): Dictionary of collected criteria. Defaults to None.
#         session (dict, optional): The user's session data for additional context. Defaults to None.
    
#     Returns:
#         list: List of structured recommendation objects with text, images, etc.
#     """
#     # Prepare the search query with all available criteria
#     if criteria:
#         criteria_terms = [value for key, value in criteria.items() if value and key != "other"]
#         if criteria_terms:
#             enhanced_query = f"{' '.join(criteria_terms)} {user_query}"
#         else:
#             enhanced_query = user_query
#     else:
#         enhanced_query = user_query
        
#     print(f"Searching for: {enhanced_query}")
    
#     # Get relevant products
#     retrieved_products = retrieve_relevant_products(enhanced_query, top_k=3)

#     if retrieved_products.empty:
#         return []

#     # Create a formatted product list for AI to process
#     product_list = "\n\n".join(
#         [
#             f"*Name:* {row['name']}\n"
#             f"*Description:* {row['description']}\n"
#             f"*Price:* ${row['price']}\n"
#             f"*Color:* {row['color']}\n"
#             for i, row in retrieved_products.iterrows()
#         ]
#     )

#     # Create conversation context to help with personalization
#     conversation_context = ""
#     if session and "conversation_history" in session:
#         # Get the last 3 user messages for context
#         user_messages = [msg["content"] for msg in session["conversation_history"] 
#                         if msg["role"] == "user"][-3:]
#         if user_messages:
#             conversation_context = "Recent conversation:\n" + "\n".join(user_messages)

#     # Construct the AI prompt with enhanced context
#     prompt = f"""
# User Query: "{user_query}"
# User Preferences: {criteria if criteria else 'Not specified'}
# {conversation_context}

# The following fashion products were retrieved as the most relevant matches:

# {product_list}

# Act as a fashion AI assistant. For each product:
# 1. Explain why it matches the user's preferences and query
# 2. Highlight key features that make it suitable for their needs
# 3. Provide 1-2 brief styling tips or suggestions
# 4. If appropriate, mention occasions where this would work well

# Be conversational and personalized. Respond with a brief paragraph (3-5 sentences max) for each product.
# Don't start with phrases like "This product..." or "Here we have...".
# Don't repeat the product name verbatim.
# """

#     # Generate response with Ollama
#     try:
#         response = ollama.chat(
#             model="mistral", messages=[{"role": "user", "content": prompt}]
#         )
#         response_text = response["message"]["content"]

#         # Split the AI response into separate product explanations
#         explanations = response_text.split("\n\n")  # Splitting paragraphs for individual products
        
#         # Format structured output for the chat
#         structured_recommendations = []
#         for (index, row), explanation in zip(retrieved_products.iterrows(), explanations[:len(retrieved_products)]):
#             # Clean up the explanation - remove product name if it appears at the start
#             clean_explanation = explanation.strip()
#             product_name_lower = row['name'].lower()
            
#             # If explanation starts with the product name, remove it
#             if clean_explanation.lower().startswith(product_name_lower[:20]):
#                 clean_explanation = clean_explanation[len(product_name_lower):].strip()
            
#             # Format the product card with emoji for visual appeal
#             structured_recommendations.append(
#                 {
#                     "text": f"**{row['name']}**\n\n💰 **${row['price']}** | 🎨 {row['color']}\n\n{clean_explanation}\n\n🔗 [View Product]({row['url']})",
#                     "image": row["image link"],
#                     "images": row["images"]
#                 }
#             )

#         return structured_recommendations  # Return structured list of text + images
#     except Exception as e:
#         print(f"Error generating recommendations: {e}")
#         # Fallback without Ollama explanations
#         structured_recommendations = []
#         for index, row in retrieved_products.iterrows():
#             structured_recommendations.append(
#                 {
#                     "text": f"**{row['name']}**\n\n💰 **${row['price']}** | 🎨 {row['color']}\n\n{row['description'][:150]}...\n\n🔗 [View Product]({row['url']})",
#                     "image": row["image link"],
#                     "images": row["images"]
#                 }
#             )
#         return structured_recommendations

In [227]:
# Fix for the "sequence item 1: expected str instance, list found" error
# Add this to the chat_fashion_assistant function

def generate_rag_response(user_query, criteria=None, session=None):
    """
    Uses Ollama to generate structured product recommendations with explanations.
    Fixed to handle list values properly.
    """
    # Prepare the search query with all available criteria
    if criteria:
        # FIX: Convert any non-string criteria values to strings before joining
        criteria_terms = []
        for key, value in criteria.items():
            if key != "other" and value is not None:
                # Handle different value types
                if isinstance(value, str):
                    criteria_terms.append(value)
                elif isinstance(value, list):
                    # Convert list to string by joining
                    criteria_terms.append(" ".join(str(item) for item in value))
                elif isinstance(value, dict):
                    # Extract values from dict
                    criteria_terms.append(" ".join(str(item) for item in value.values() if item))
                else:
                    # Convert other types to string
                    criteria_terms.append(str(value))
                    
        if criteria_terms:
            enhanced_query = f"{' '.join(criteria_terms)} {user_query}"
        else:
            enhanced_query = user_query
    else:
        enhanced_query = user_query
        
    print(f"Searching for: {enhanced_query}")
    
    # Get relevant products
    retrieved_products = retrieve_relevant_products(enhanced_query, top_k=3)

    if retrieved_products.empty:
        return []

    # Create a formatted product list for AI to process
    product_list = "\n\n".join(
        [
            f"*Name:* {row['name']}\n"
            f"*Description:* {row['description']}\n"
            f"*Price:* ${row['price']}\n"
            f"*Color:* {row['color']}\n"
            for i, row in retrieved_products.iterrows()
        ]
    )

    # Create conversation context to help with personalization
    conversation_context = ""
    if session and "conversation_history" in session:
        # Get the last 3 user messages for context
        user_messages = [msg["content"] for msg in session["conversation_history"] 
                        if msg["role"] == "user"][-3:]
        if user_messages:
            conversation_context = "Recent conversation:\n" + "\n".join(user_messages)

    # Construct the AI prompt with enhanced context
    prompt = f"""
User Query: "{user_query}"
User Preferences: {str(criteria) if criteria else 'Not specified'}
{conversation_context}

The following fashion products were retrieved as the most relevant matches:

{product_list}

Act as a fashion AI assistant. For each product:
1. Explain why it matches the user's preferences and query
2. Highlight key features that make it suitable for their needs
3. Provide 1-2 brief styling tips or suggestions
4. If appropriate, mention occasions where this would work well

Be conversational and personalized. Respond with a brief paragraph (3-5 sentences max) for each product.
Don't start with phrases like "This product..." or "Here we have...".
Don't repeat the product name verbatim.
"""

    # Generate response with Ollama
    try:
        response = ollama.chat(
            model="mistral", messages=[{"role": "user", "content": prompt}]
        )
        response_text = response["message"]["content"]

        # Split the AI response into separate product explanations
        explanations = response_text.split("\n\n")  # Splitting paragraphs for individual products
        
        # Format structured output for the chat
        structured_recommendations = []
        for (index, row), explanation in zip(retrieved_products.iterrows(), explanations[:len(retrieved_products)]):
            # Clean up the explanation - remove product name if it appears at the start
            clean_explanation = explanation.strip()
            product_name_lower = str(row['name']).lower()
            
            # If explanation starts with the product name, remove it
            if clean_explanation.lower().startswith(product_name_lower[:20]):
                clean_explanation = clean_explanation[len(product_name_lower):].strip()
            
            # Format the product card with emoji for visual appeal
            structured_recommendations.append(
                {
                    "text": f"**{row['name']}**\n\n💰 **${row['price']}** | 🎨 {row['color']}\n\n{clean_explanation}\n\n🔗 [View Product]({row['url']})",
                    "image": row["image link"],
                    "images": row["images"]
                }
            )

        return structured_recommendations  # Return structured list of text + images
    except Exception as e:
        print(f"Error generating recommendations: {e}")
        import traceback
        traceback.print_exc()
        # Fallback without Ollama explanations
        structured_recommendations = []
        for index, row in retrieved_products.iterrows():
            structured_recommendations.append(
                {
                    "text": f"**{row['name']}**\n\n💰 **${row['price']}** | 🎨 {row['color']}\n\n{row['description'][:150]}...\n\n🔗 [View Product]({row['url']})",
                    "image": row["image link"],
                    "images": row["images"]
                }
            )
        return structured_recommendations

Step 6: 
- Chat function for Gradio

In [228]:
import ast

# def chat_fashion_assistant(user_input, history):
#     """
#     Processes user query and returns structured product recommendations with AI-generated explanations.
#     """
#     #Get AI-generated recommendations
#     recommendations = generate_rag_response(user_input)

#     if not recommendations:
#         return [{"role": "assistant", "content": "No matching products found."}]

#     #Create structured response list
#     response_list = []
#     for rec in recommendations:
#         images = ast.literal_eval(rec["images"])
        
#         response_list.append({"role": "assistant", "content": rec["text"]})  # AI explanation
#         response_list.append({"role": "assistant", "content": gr.Gallery(images[:4], 
#                                                                         columns=4, 
#                                                                         rows=1, 
#                                                                         object_fit="cover", 
#                                                                         height="automatic",
#                                                                         allow_preview=True)
#                                                                         }) 

#         response_list.append({"role": "assistant", "content": "\n\n\n\n\n\n"})  # Adds a horizontal divider and spacing

#         # response_list.append(gr.Image(rec["image"]))  # Product image

#     return response_list  #Returning structured chat messages


In [229]:
# '''
# STEP 6
# '''

# import ast
# import json
# from collections import defaultdict

# # Track conversation state
# user_sessions = defaultdict(lambda: {
#     "conversation_history": [],
#     "criteria_collected": {},
#     "stage": "initial",
#     "recommendations_shown": False
# })

# def chat_fashion_assistant(user_input, history, session_id="default"):
#     """
#     AI-powered conversational shopping assistant that maintains conversation state
#     and provides personalized product recommendations.
    
#     Args:
#         user_input (str): The user's message
#         history (list): The chat history
#         session_id (str, optional): Unique identifier for the user session. Defaults to "default".
    
#     Returns:
#         list: List of response messages with text and visual content
#     """
#     # Get or create session
#     session = user_sessions[session_id]
    
#     # Special case for conversation restart
#     restart_phrases = ["start over", "restart", "reset", "new search", "start again"]
#     if any(phrase in user_input.lower() for phrase in restart_phrases):
#         # Reset the session
#         session["criteria_collected"] = {}
#         session["stage"] = "initial"
#         session["recommendations_shown"] = False
#         session["conversation_history"] = []
#         return [{"role": "assistant", "content": "Let's start fresh! What are you looking for today?"}]
    
#     # Add user message to conversation history
#     session["conversation_history"].append({"role": "user", "content": user_input})
    
#     # Have AI analyze the message and context
#     try:
#         # Create a context-aware prompt for the AI
#         prompt = f"""
# You are a fashion shopping assistant helping a customer find products. Analyze their latest message and previous conversation to:

# 1. Extract shopping criteria (if any)
# 2. Determine what the conversation needs next
# 3. Format your response as JSON

# Previous conversation:
# {json.dumps(session["conversation_history"][:-1])[:1000] if len(session["conversation_history"]) > 1 else "This is the start of the conversation."}

# Current criteria collected:
# {json.dumps(session["criteria_collected"])}

# Current stage: {session["stage"]}
# Recommendations shown: {"Yes" if session["recommendations_shown"] else "No"}

# Latest user message: "{user_input}"

# Respond with JSON only, in this format:
# {{
#   "extracted_criteria": {{
#     "item_type": "detected item or null",
#     "style": "detected style or null",
#     "color": "detected color or null", 
#     "occasion": "detected occasion or null",
#     "price_range": "detected price range or null",
#     "other": "any other important criteria detected"
#   }},
#   "action": "one of: greet, ask_for_criteria, search_products, show_more, restart, answer_question",
#   "missing_criterion": "most important missing criterion to ask about or null",
#   "question": "follow-up question to ask (if applicable) or null",
#   "understanding": "brief summary of what you understand about their needs",
#   "ready_to_search": true/false,
#   "search_query": "optimized search query based on all criteria and context"
# }}

# Important: 
# - The extracted_criteria should ONLY include values you're confident about from the current message or previous context
# - Do NOT invent criteria not mentioned by the user
# - For action, use "search_products" only when you have enough criteria OR the user is clearly asking to see products
# - For search_query, optimize the terms for product search (remove filler words, focus on key features)
# """

#         # Send to Ollama for analysis
#         response = ollama.chat(
#             model="mistral", 
#             messages=[{"role": "user", "content": prompt}]
#         )
        
#         # Parse the JSON response
#         analysis_text = response["message"]["content"]
#         # Find the JSON part in case there's additional text
#         json_start = analysis_text.find('{')
#         json_end = analysis_text.rfind('}') + 1
        
#         if json_start >= 0 and json_end > json_start:
#             json_part = analysis_text[json_start:json_end]
#             analysis = json.loads(json_part)
#         else:
#             # Fallback if JSON parsing fails
#             raise ValueError("Could not extract valid JSON from the response")
        
#     except Exception as e:
#         print(f"Error analyzing message: {e}")
#         # Fallback analysis if there's an error
#         analysis = {
#             "extracted_criteria": {},
#             "action": "search_products" if "show me" in user_input.lower() else "ask_for_criteria",
#             "missing_criterion": "item_type",
#             "question": "What type of clothing are you looking for?",
#             "understanding": "I'm trying to understand what you're looking for.",
#             "ready_to_search": False,
#             "search_query": user_input
#         }
    
#     # Update session with extracted criteria
#     for criterion, value in analysis["extracted_criteria"].items():
#         if value and value.lower() not in ("null", "none"):
#             session["criteria_collected"][criterion] = value
    
#     # Handle different conversation actions
#     if analysis["action"] == "greet":
#         session["stage"] = "collecting"
#         return [{"role": "assistant", "content": f"Hi! I'd be happy to help you find the perfect outfit. {analysis.get('question', 'What type of clothing are you looking for today?')}"}]
    
#     elif analysis["action"] == "ask_for_criteria":
#         session["stage"] = "collecting"
#         question = analysis.get("question") or f"Can you tell me what kind of {analysis.get('missing_criterion', 'item')} you're looking for?"
#         return [{"role": "assistant", "content": question}]
    
#     elif analysis["action"] == "search_products" or analysis["action"] == "show_more":
#         session["stage"] = "searching"
#         # Use the existing retrieve_relevant_products and generate_rag_response functions
#         search_query = analysis.get("search_query") or user_input
        
#         # Get AI-generated recommendations
#         recommendations = generate_rag_response(search_query, session["criteria_collected"], session)
        
#         if not recommendations:
#             # No products found - ask if user wants to broaden search
#             session["recommendations_shown"] = True
#             return [{"role": "assistant", "content": "I couldn't find any products matching your criteria. Could you be more general or try different options?"}]
        
#         # Create structured response list
#         response_list = []
        
#         # First, summarize what we understood from their requirements
#         criteria_display = {
#             "item_type": "Item", 
#             "style": "Style", 
#             "color": "Color", 
#             "occasion": "Occasion", 
#             "price_range": "Price",
#             "other": "Other"
#         }
        
#         criteria_summary = ", ".join([f"{criteria_display[k]}: {v}" for k, v in session["criteria_collected"].items() 
#                                       if v and k in criteria_display])
        
#         if criteria_summary:
#             response_list.append({"role": "assistant", "content": f"Here are some recommendations based on your preferences ({criteria_summary}):"})
#         else:
#             response_list.append({"role": "assistant", "content": "Here are some recommendations based on your request:"})
        
#         # Add products with images and descriptions
#         for rec in recommendations:
#             try:
#                 images = ast.literal_eval(rec["images"])
#             except:
#                 # Fallback if images can't be parsed
#                 images = [rec["image"]]
            
#             response_list.append({"role": "assistant", "content": rec["text"]})  # Product with AI explanation
#             response_list.append({"role": "assistant", "content": gr.Gallery(images[:4], 
#                                                                           columns=4, 
#                                                                           rows=1, 
#                                                                           object_fit="cover", 
#                                                                           height="automatic",
#                                                                           allow_preview=True)
#                                                                          }) 
    
#             response_list.append({"role": "assistant", "content": "\n\n\n\n\n\n"})  # Adds spacing
    
#         # Add follow-up prompt
#         response_list.append({"role": "assistant", "content": "Would you like to see more options, or should I help you find something else?"})
        
#         # Update session
#         session["recommendations_shown"] = True
#         session["stage"] = "recommending"
        
#         # Add assistant response to conversation history
#         session["conversation_history"].append({"role": "assistant", "content": "I showed some product recommendations based on the user's criteria."})
        
#         return response_list
    
#     elif analysis["action"] == "restart":
#         # Reset the session (redundant with the check at the beginning, but keeping for completeness)
#         session["criteria_collected"] = {}
#         session["stage"] = "collecting" 
#         session["recommendations_shown"] = False
#         session["conversation_history"] = []
#         return [{"role": "assistant", "content": "Let's start fresh! What are you looking for today?"}]
    
#     elif analysis["action"] == "answer_question":
#         # If the AI detects a question not related to product search
#         if "question" in analysis and analysis["question"]:
#             return [{"role": "assistant", "content": analysis["question"]}]
#         else:
#             return [{"role": "assistant", "content": "I'm here to help you find fashion products. What kind of item are you looking for?"}]
    
#     # Default fallback response if analysis doesn't yield actionable results
#     return [{"role": "assistant", "content": "I'm not sure I understood. Could you tell me what kind of clothing or fashion item you're looking for?"}]

In [230]:
def analyze_image_with_llava(image_path, user_query=None):
    """
    Uses LLaVA to generate a description of the clothing item in the image,
    focusing on specific items mentioned in the user's query.
    
    Args:
        image_path (str): Path to the uploaded image
        user_query (str, optional): The user's text query to focus the analysis
        
    Returns:
        tuple: (description, keywords) - The focused description and extracted keywords
    """
    import os
    import subprocess
    import shlex
    import re
    import tempfile
    
    print(f"[DEBUG] Starting image analysis for: {image_path}")
    if user_query:
        print(f"[DEBUG] Query-aware analysis with: '{user_query}'")
    
    # Extract specific item mention from the query
    target_item = None
    if user_query:
        # Common clothing items to look for in the query
        clothing_items = [
            "jacket", "coat", "dress", "shirt", "pants", "jeans", "sweater", 
            "hoodie", "skirt", "blazer", "suit", "top", "shorts", "blouse", 
            "t-shirt", "tee", "shoes", "boots", "sneakers", "hat", "scarf",
            "bag", "purse", "backpack", "accessory", "jewelry", "watch",
            "glasses", "sunglasses", "outfit", "look", "style", "fashion"
        ]
        
        # Extract the specific item the user is interested in
        for item in clothing_items:
            if item in user_query.lower():
                target_item = item
                print(f"[DEBUG] Detected target item in query: '{target_item}'")
                break
    
    # Use the exact same LLaVA paths as in Step 2
    LLAVA_CLI_PATH = "/Users/glennsuristio/Documents/Projects/dressAI/llama.cpp/llama-llava-cli"
    LLAVA_MODEL_PATH = "/Users/glennsuristio/Documents/Projects/dressAI/llava-v1.6-mistral-7b/Mistral-7B-Instruct-v0.2-F32-Q4_K_M.gguf"
    MM_PROJ_PATH = "/Users/glennsuristio/Documents/Projects/dressAI/llama.cpp/vit/mmproj-model-f16.gguf"
    
    # Temporary file for LLaVA output
    output_file = os.path.join(tempfile.gettempdir(), "llava_output.txt")
    print(f"[DEBUG] LLaVA output will be saved to: {output_file}")
    
    # Craft a more focused prompt based on the user's query
    if target_item:
        # Focus on the specific item the user mentioned
        prompt = f"""
USER:
Describe only the {target_item} in this image in detail. Focus on:
- Style, design, and unique features of the {target_item}
- Color and pattern details 
- Material and texture appearance
- Fit and silhouette
- Any distinctive elements or embellishments

Ignore other clothing items or accessories unless they're part of the {target_item}.
Then provide a list of 5-7 keywords that best describe this {target_item}, prefixed with "KEYWORDS:".

A:
"""
    else:
        # Generic clothing description if no specific item was mentioned
        prompt = f"""
USER:
Describe the most prominent clothing item in this image in detail. Focus on:
- Identify the main garment (e.g., jacket, dress, pants)
- Style, design, and unique features
- Color and pattern details
- Material and texture appearance
- Fit and silhouette
- Any distinctive elements or embellishments

Then provide a list of 5-7 keywords that best describe this item, prefixed with "KEYWORDS:".

A:
"""

    # Run LLaVA command
    try:
        command = f'{LLAVA_CLI_PATH} -m {LLAVA_MODEL_PATH} --mmproj {MM_PROJ_PATH} --image {shlex.quote(image_path)} -c 4096 -p "{prompt}" > {output_file}'
        print(f"[DEBUG] Executing LLaVA command")
        
        process = subprocess.run(command, shell=True, capture_output=True, text=True)
        
        # Check if the command succeeded
        if process.returncode != 0:
            print(f"[ERROR] LLaVA command failed with return code {process.returncode}")
            print(f"[ERROR] stderr: {process.stderr}")
            return f"A clothing item" + (f" (possibly a {target_item})" if target_item else ""), ["clothing"]
        
        print(f"[DEBUG] LLaVA command completed successfully")
        
        # Read the output file
        with open(output_file, 'r') as f:
            output_text = f.read()
            
        print(f"[DEBUG] LLaVA output file size: {len(output_text)} characters")
        
        # Extract the description part
        description_lines = []
        capture = False
        
        for line in output_text.split('\n'):
            if 'encode_image_with_clip: image encoded' in line:
                capture = True
                print(f"[DEBUG] Found marker line for description start")
                continue
            
            if capture and not line.startswith('['):
                description_lines.append(line)
        
        description = ' '.join(description_lines).strip()
        print(f"[DEBUG] Extracted description ({len(description)} chars): {description[:100]}...")
        
        # Extract keywords from the description
        keywords = []
        if "KEYWORDS:" in description:
            print(f"[DEBUG] Found KEYWORDS section in output")
            # Get the part after "KEYWORDS:"
            keyword_section = description.split("KEYWORDS:")[1].strip()
            # Split by commas or spaces if no commas
            if "," in keyword_section:
                keywords = [k.strip().lower() for k in keyword_section.split(",")]
            else:
                keywords = re.findall(r'\b[A-Za-z]+\b', keyword_section.lower())
            
            # Clean up the description to remove the keywords section
            description = description.split("KEYWORDS:")[0].strip()
            print(f"[DEBUG] Successfully extracted {len(keywords)} keywords: {keywords}")
            
            # Add the target item to keywords if it's not already there
            if target_item and target_item not in keywords:
                keywords.insert(0, target_item)
        else:
            print(f"[DEBUG] No KEYWORDS section found, extracting from description")
            # If no keywords section, extract important words from description
            clothing_terms = ["dress", "jacket", "shirt", "pants", "coat", "blouse", "sweater", 
                             "jeans", "skirt", "hoodie", "blazer", "suit", "top", "shorts"]
            color_terms = ["black", "white", "red", "blue", "green", "yellow", "purple", 
                          "pink", "orange", "grey", "gray", "brown", "navy", "beige"]
            style_terms = ["casual", "formal", "elegant", "vintage", "modern", "classic", 
                          "sporty", "bohemian", "minimalist", "business", "trendy"]
            
            # Look for these terms in the description
            words = re.findall(r'\b[A-Za-z]+\b', description.lower())
            for word in words:
                if (word in clothing_terms or word in color_terms or word in style_terms) and word not in keywords:
                    keywords.append(word)
            
            # Ensure the target item is included
            if target_item and target_item not in keywords:
                keywords.insert(0, target_item)
                
            # Limit to most relevant keywords
            keywords = keywords[:7]
            print(f"[DEBUG] Extracted {len(keywords)} keywords from description: {keywords}")
        
        # If we couldn't extract a good description or keywords, provide a fallback
        if not description or len(description) < 20:
            print(f"[DEBUG] Description too short, using fallback")
            return f"A clothing item" + (f" (possibly a {target_item})" if target_item else ""), ["clothing"]
            
        print(f"[DEBUG] LLaVA analysis completed successfully ✅")
        return description, keywords
        
    except Exception as e:
        print(f"[ERROR] Exception during image analysis: {e}")
        import traceback
        traceback.print_exc()
        # Provide a fallback in case of error
        return f"A clothing item" + (f" (possibly a {target_item})" if target_item else ""), ["clothing"]

In [231]:
# Track conversation state
user_sessions = defaultdict(lambda: {
    "conversation_history": [],
    "criteria_collected": {},
    "stage": "initial",
    "recommendations_shown": False,
    "image_analysis": None
})

In [232]:
# '''
# STEP 6: Updated Chat Fashion Assistant
# '''

# import ast
# import json
# from collections import defaultdict

# # Track conversation state
# user_sessions = defaultdict(lambda: {
#     "conversation_history": [],
#     "criteria_collected": {},
#     "stage": "initial",
#     "recommendations_shown": False,
#     "image_analysis": None
# })

# def chat_fashion_assistant(user_input, image=None, history=None, session_id="default"):
#     """
#     AI-powered conversational shopping assistant that accepts both text and image inputs.
    
#     Args:
#         user_input (str): The user's text message
#         image (Image, optional): Uploaded image for visual search. Defaults to None.
#         history (list, optional): Chat history. Defaults to None.
#         session_id (str, optional): Unique session identifier. Defaults to "default".
    
#     Returns:
#         list: List of response messages
#     """
#     # Initialize history if None
#     if history is None:
#         history = []
    
#     # Get or create session
#     session = user_sessions[session_id]
    
#     # Handle special restart requests
#     restart_phrases = ["start over", "restart", "reset", "new search", "start again"]
#     if user_input and any(phrase in user_input.lower() for phrase in restart_phrases):
#         # Reset the session
#         session["criteria_collected"] = {}
#         session["stage"] = "initial"
#         session["recommendations_shown"] = False
#         session["conversation_history"] = []
#         session["image_analysis"] = None
#         return [{"role": "assistant", "content": "Let's start fresh! What are you looking for today? You can describe it or upload an image."}]
    
#     # Handle image upload if present
#     search_query = user_input if user_input else ""
#     image_description = None
    
#     if image is not None:
#         try:
#             # Process the image
#             import os
#             import tempfile
#             from PIL import Image as PILImage
            
#             print(f"[DEBUG] Image detected in chat input")
            
#             # Save the uploaded image to a temporary file
#             temp_dir = tempfile.gettempdir()
#             temp_img_path = os.path.join(temp_dir, f"upload_{os.urandom(4).hex()}.jpg")
            
#             # Handle different image input types
#             if isinstance(image, str):  # Path to image
#                 image_path = image
#                 print(f"[DEBUG] Image is a file path: {image_path}")
#             elif hasattr(image, 'save'):  # PIL Image
#                 image.save(temp_img_path)
#                 image_path = temp_img_path
#                 print(f"[DEBUG] Saved PIL image to: {image_path}")
#             else:
#                 # Skip image processing if we can't handle the type
#                 print(f"[DEBUG] Unsupported image type: {type(image)}")
#                 image_path = None
            
#             if image_path and os.path.exists(image_path):
#                 print(f"[DEBUG] Calling analyze_image_with_llava with path: {image_path}")
#                 # Call the analyze_image_with_llava function
#                 image_description, keywords = analyze_image_with_llava(image_path)
                
#                 print(f"[DEBUG] Image analysis complete. Description: {image_description[:50]}...")
#                 print(f"[DEBUG] Extracted keywords: {keywords}")
                
#                 # Store in session
#                 session["image_analysis"] = {
#                     "description": image_description,
#                     "keywords": keywords,
#                     "image_path": image_path
#                 }
                
#                 # Enhance the search query with keywords
#                 if keywords:
#                     keyword_str = " ".join(keywords)
#                     if search_query:
#                         search_query = f"{search_query} {keyword_str}"
#                     else:
#                         search_query = keyword_str
#                     print(f"[DEBUG] Enhanced search query: {search_query}")
                
#                 # If we just have an image with no text query, we'll acknowledge the image
#                 if not user_input or user_input.strip() == "":
#                     # Add initial response about the image
#                     image_response = {"role": "assistant", "content": f"I can see the item in your image. It looks like {image_description[:150]}..."}
#                     follow_up = {"role": "assistant", "content": "Is there anything specific about this item you're looking for? Or shall I find similar products?"}
                    
#                     # Add this interaction to conversation history
#                     session["conversation_history"].append({"role": "user", "content": f"[Uploaded an image]"})
#                     session["conversation_history"].append({"role": "assistant", "content": image_response["content"]})
#                     session["conversation_history"].append({"role": "assistant", "content": follow_up["content"]})
                    
#                     # Update criteria from image analysis
#                     for keyword in keywords:
#                         # Try to identify item type
#                         item_types = ["jacket", "dress", "shirt", "pants", "jeans", "skirt", "sweater", "coat", 
#                                      "blouse", "hoodie", "blazer", "suit", "shorts", "top"]
#                         if keyword.lower() in item_types and "item_type" not in session["criteria_collected"]:
#                             session["criteria_collected"]["item_type"] = keyword
                            
#                         # Try to identify color
#                         colors = ["black", "white", "red", "blue", "green", "yellow", "purple", "pink", 
#                                  "orange", "grey", "gray", "brown", "navy", "beige"]
#                         if keyword.lower() in colors and "color" not in session["criteria_collected"]:
#                             session["criteria_collected"]["color"] = keyword
                    
#                     return [image_response, follow_up]
#         except Exception as e:
#             print(f"[ERROR] Exception during image processing: {e}")
#             # Continue without image analysis
    
#     # Add user message to conversation history if there is one
#     if user_input and user_input.strip():
#         session["conversation_history"].append({"role": "user", "content": user_input})
    
#     # Have AI analyze the message and context
#     try:
#         # Create a context-aware prompt for the AI
#         prompt = f"""
# You are a fashion shopping assistant helping a customer find products. Analyze their latest message and previous conversation to:

# 1. Extract shopping criteria (if any)
# 2. Determine what the conversation needs next
# 3. Format your response as JSON

# Previous conversation:
# {json.dumps(session["conversation_history"][:-1])[:1000] if len(session["conversation_history"]) > 1 else "This is the start of the conversation."}

# Current criteria collected:
# {json.dumps(session["criteria_collected"])}

# Current stage: {session["stage"]}
# Recommendations shown: {"Yes" if session["recommendations_shown"] else "No"}
# {"Image analysis: " + json.dumps(session["image_analysis"]) if session.get("image_analysis") else "No image uploaded"}

# Latest user message: "{user_input}"

# Respond with JSON only, in this format:
# {{
#   "extracted_criteria": {{
#     "item_type": "detected item or null",
#     "style": "detected style or null",
#     "color": "detected color or null", 
#     "occasion": "detected occasion or null",
#     "price_range": "detected price range or null",
#     "other": "any other important criteria detected"
#   }},
#   "action": "one of: greet, ask_for_criteria, search_products, show_more, restart, answer_question",
#   "missing_criterion": "most important missing criterion to ask about or null",
#   "question": "follow-up question to ask (if applicable) or null",
#   "understanding": "brief summary of what you understand about their needs",
#   "ready_to_search": true/false,
#   "search_query": "optimized search query based on all criteria and context"
# }}

# Important: 
# - The extracted_criteria should ONLY include values you're confident about from the current message or previous context
# - Do NOT invent criteria not mentioned by the user
# - For action, use "search_products" only when you have enough criteria OR the user is clearly asking to see products
# - For search_query, optimize the terms for product search (remove filler words, focus on key features)
# """

#         # Send to Ollama for analysis
#         response = ollama.chat(
#             model="mistral", 
#             messages=[{"role": "user", "content": prompt}]
#         )
        
#         # Parse the JSON response
#         analysis_text = response["message"]["content"]
#         # Find the JSON part in case there's additional text
#         json_start = analysis_text.find('{')
#         json_end = analysis_text.rfind('}') + 1
        
#         if json_start >= 0 and json_end > json_start:
#             json_part = analysis_text[json_start:json_end]
#             analysis = json.loads(json_part)
#         else:
#             # Fallback if JSON parsing fails
#             raise ValueError("Could not extract valid JSON from the response")
        
#     except Exception as e:
#         print(f"[ERROR] Error analyzing message: {e}")
#         # Fallback analysis if there's an error
#         analysis = {
#             "extracted_criteria": {},
#             "action": "search_products" if "show me" in user_input.lower() or image is not None else "ask_for_criteria",
#             "missing_criterion": "item_type",
#             "question": "What type of clothing are you looking for?",
#             "understanding": "I'm trying to understand what you're looking for.",
#             "ready_to_search": False,
#             "search_query": search_query
#         }
    
#     # Update session with extracted criteria
#     for criterion, value in analysis["extracted_criteria"].items():
#         if value and value.lower() not in ("null", "none"):
#             session["criteria_collected"][criterion] = value
    
#     # Handle different conversation actions
#     if analysis["action"] == "greet":
#         session["stage"] = "collecting"
#         return [{"role": "assistant", "content": f"Hi! I'd be happy to help you find the perfect outfit. You can describe what you're looking for or upload an image. {analysis.get('question', 'What type of clothing are you looking for today?')}"}]
    
#     elif analysis["action"] == "ask_for_criteria":
#         session["stage"] = "collecting"
#         question = analysis.get("question") or f"Can you tell me what kind of {analysis.get('missing_criterion', 'item')} you're looking for?"
#         return [{"role": "assistant", "content": question}]
    
#     elif analysis["action"] == "search_products" or analysis["action"] == "show_more":
#         session["stage"] = "searching"
#         # Use the existing retrieve_relevant_products and generate_rag_response functions
#         search_query = analysis.get("search_query") or search_query
        
#         # If we have image analysis, incorporate it into the search (if not already in search_query)
#         if session.get("image_analysis") and session["image_analysis"].get("keywords"):
#             # Check if keywords are already in the query
#             keyword_present = False
#             for keyword in session["image_analysis"]["keywords"]:
#                 if keyword.lower() in search_query.lower():
#                     keyword_present = True
#                     break
            
#             if not keyword_present:
#                 keyword_str = " ".join(session["image_analysis"]["keywords"])
#                 search_query = f"{search_query} {keyword_str}"
#                 print(f"[DEBUG] Added image keywords to search query: {search_query}")
        
#         # Get AI-generated recommendations
#         print(f"[DEBUG] Calling generate_rag_response with query: {search_query}")
#         recommendations = generate_rag_response(search_query, session["criteria_collected"], session)
        
#         if not recommendations:
#             # No products found - ask if user wants to broaden search
#             session["recommendations_shown"] = True
#             return [{"role": "assistant", "content": "I couldn't find any products matching your criteria. Could you be more general or try different options?"}]
        
#         # Create structured response list
#         response_list = []
        
#         # First, summarize what we understood from their requirements
#         criteria_display = {
#             "item_type": "Item", 
#             "style": "Style", 
#             "color": "Color", 
#             "occasion": "Occasion", 
#             "price_range": "Price",
#             "other": "Other"
#         }
        
#         criteria_summary = ", ".join([f"{criteria_display[k]}: {v}" for k, v in session["criteria_collected"].items() 
#                                       if v and k in criteria_display])
        
#         # If we have image analysis, mention it
#         if session.get("image_analysis"):
#             if criteria_summary:
#                 response_list.append({"role": "assistant", "content": f"Based on your image and preferences ({criteria_summary}), here are some recommendations:"})
#             else:
#                 response_list.append({"role": "assistant", "content": f"Based on your image, here are some similar products:"})
#         elif criteria_summary:
#             response_list.append({"role": "assistant", "content": f"Here are some recommendations based on your preferences ({criteria_summary}):"})
#         else:
#             response_list.append({"role": "assistant", "content": "Here are some recommendations based on your request:"})
        
#         # Add products with images and descriptions
#         for rec in recommendations:
#             try:
#                 images = ast.literal_eval(rec["images"])
#             except:
#                 # Fallback if images can't be parsed
#                 images = [rec["image"]]
            
#             response_list.append({"role": "assistant", "content": rec["text"]})  # Product with AI explanation
#             response_list.append({"role": "assistant", "content": gr.Gallery(images[:4], 
#                                                                           columns=4, 
#                                                                           rows=1, 
#                                                                           object_fit="cover", 
#                                                                           height="automatic",
#                                                                           allow_preview=True)
#                                                                          }) 
    
#             response_list.append({"role": "assistant", "content": "\n\n\n\n\n\n"})  # Adds spacing
    
#         # Add follow-up prompt
#         response_list.append({"role": "assistant", "content": "Would you like to see more options, or should I help you find something else?"})
        
#         # Update session
#         session["recommendations_shown"] = True
#         session["stage"] = "recommending"
        
#         # Add assistant response to conversation history
#         session["conversation_history"].append({"role": "assistant", "content": "I showed some product recommendations based on the user's criteria."})
        
#         return response_list
    
#     elif analysis["action"] == "restart":
#         # Reset the session (redundant with the check at the beginning, but keeping for completeness)
#         session["criteria_collected"] = {}
#         session["stage"] = "collecting" 
#         session["recommendations_shown"] = False
#         session["conversation_history"] = []
#         session["image_analysis"] = None
#         return [{"role": "assistant", "content": "Let's start fresh! What are you looking for today? You can describe it or upload an image."}]
    
#     elif analysis["action"] == "answer_question":
#         # If the AI detects a question not related to product search
#         if "question" in analysis and analysis["question"]:
#             return [{"role": "assistant", "content": analysis["question"]}]
#         else:
#             return [{"role": "assistant", "content": "I'm here to help you find fashion products. What kind of item are you looking for?"}]
    
#     # Default fallback response if analysis doesn't yield actionable results
#     return [{"role": "assistant", "content": "I'm not sure I understood. Could you tell me what kind of clothing or fashion item you're looking for? You can also upload an image if you have one."}]

In [233]:
def chat_fashion_assistant(user_input, image=None, history=None, session_id="default"):
    """
    AI-powered conversational shopping assistant with improved error handling.
    
    Args:
        user_input (str): The user's text message
        image (Image, optional): Uploaded image for visual search. Defaults to None.
        history (list, optional): Chat history. Defaults to None.
        session_id (str, optional): Unique session identifier. Defaults to "default".
    
    Returns:
        list: List of response messages
    """
    # Initialize history if None
    if history is None:
        history = []
    
    # Get or create session
    session = user_sessions[session_id]
    
    # Handle special restart requests
    restart_phrases = ["start over", "restart", "reset", "new search", "start again"]
    if user_input and any(phrase in user_input.lower() for phrase in restart_phrases):
        # Reset the session
        session["criteria_collected"] = {}
        session["stage"] = "initial"
        session["recommendations_shown"] = False
        session["conversation_history"] = []
        session["image_analysis"] = None
        return [{"role": "assistant", "content": "Let's start fresh! What are you looking for today? You can describe it or upload an image."}]
    
    # Handle empty inputs gracefully
    if not user_input.strip() and image is None:
        # If there's truly no input, provide a greeting/help message
        return [{"role": "assistant", "content": "Hi there! I can help you find clothing items. What are you looking for today? You can also upload an image if you have something specific in mind."}]
    
    # Handle image upload if present
    search_query = user_input if user_input else ""
    image_description = None
    
    if image is not None:
        try:
            # Process the image
            import os
            import tempfile
            from PIL import Image as PILImage
            
            print(f"[DEBUG] Image detected in chat input")
            
            # Save the uploaded image to a temporary file if needed
            if isinstance(image, str) and os.path.exists(image):
                image_path = image
                print(f"[DEBUG] Using existing image path: {image_path}")
            else:
                # Handle PIL Image or other image types
                temp_dir = tempfile.gettempdir()
                temp_img_path = os.path.join(temp_dir, f"upload_{os.urandom(4).hex()}.jpg")
                
                if hasattr(image, 'save'):  # PIL Image
                    image.save(temp_img_path)
                    image_path = temp_img_path
                    print(f"[DEBUG] Saved PIL image to: {image_path}")
                else:
                    print(f"[DEBUG] Unsupported image type: {type(image)}")
                    image_path = None
            
            if image_path and os.path.exists(image_path):
                print(f"[DEBUG] Calling analyze_image_with_llava with path: {image_path}")
                # Call the analyze_image_with_llava function
                image_description, keywords = analyze_image_with_llava(image_path, user_input)
                
                print(f"[DEBUG] Image analysis complete. Description: {image_description[:50]}...")
                print(f"[DEBUG] Extracted keywords: {keywords}")
                
                # Store in session
                session["image_analysis"] = {
                    "description": image_description,
                    "keywords": keywords,
                    "image_path": image_path
                }
                
                # Enhance the search query with keywords
                if keywords:
                    keyword_str = " ".join(keywords)
                    if search_query:
                        search_query = f"{search_query} {keyword_str}"
                    else:
                        search_query = keyword_str
                    print(f"[DEBUG] Enhanced search query: {search_query}")
        except Exception as e:
            print(f"[ERROR] Exception during image processing: {e}")
            # Continue without image analysis
    
    # Add user message to conversation history
    # IMPORTANT: Make sure we're storing the exact message that was input, not a modified version
    # This ensures different messages remain separate
    session["conversation_history"].append({"role": "user", "content": user_input})
    
    # If we have image analysis, acknowledge it
    if image is not None and image_description:
        # Add initial response about the image
        image_response = {"role": "assistant", "content": f"I can see the item in your image. It looks like {image_description[:100]}..."}
        
        # If there's no text query, ask for more details and return
        if not user_input.strip():
            session["conversation_history"].append({"role": "assistant", "content": image_response["content"]})
            return [image_response, {"role": "assistant", "content": "Is there anything specific about this item you're looking for? Or shall I find similar products?"}]
    
    # Have AI analyze the message and context
    try:
        # Create a context-aware prompt for the AI
        prompt = f"""
You are a fashion shopping assistant helping a customer find products. Analyze their latest message and previous conversation to:

1. Extract shopping criteria (if any)
2. Determine what the conversation needs next
3. Format your response as JSON

Previous conversation:
{json.dumps(session["conversation_history"][:-1])[:1000] if len(session["conversation_history"]) > 1 else "This is the start of the conversation."}

Current criteria collected:
{json.dumps(session["criteria_collected"])}

Current stage: {session["stage"]}
Recommendations shown: {"Yes" if session["recommendations_shown"] else "No"}
{"Image analysis: " + json.dumps(session["image_analysis"]) if session.get("image_analysis") else "No image uploaded"}

Latest user message: "{user_input}"

Respond with JSON only, in this format:
{{
  "extracted_criteria": {{
    "item_type": "detected item or null",
    "style": "detected style or null",
    "color": "detected color or null", 
    "occasion": "detected occasion or null",
    "price_range": "detected price range or null",
    "other": "any other important criteria detected"
  }},
  "action": "one of: greet, ask_for_criteria, search_products, show_more, restart, answer_question",
  "missing_criterion": "most important missing criterion to ask about or null",
  "question": "follow-up question to ask (if applicable) or null",
  "understanding": "brief summary of what you understand about their needs",
  "ready_to_search": true/false,
  "search_query": "optimized search query based on all criteria and context"
}}

Important: 
- The extracted_criteria should ONLY include values you're confident about from the current message or previous context
- Do NOT invent criteria not mentioned by the user
- For action, use "search_products" only when you have enough criteria OR the user is clearly asking to see products
- For search_query, optimize the terms for product search (remove filler words, focus on key features)
"""

        # Send to Ollama for analysis
        response = ollama.chat(
            model="mistral", 
            messages=[{"role": "user", "content": prompt}]
        )
        
        # Parse the JSON response
        analysis_text = response["message"]["content"]
        # Find the JSON part in case there's additional text
        json_start = analysis_text.find('{')
        json_end = analysis_text.rfind('}') + 1
        
        if json_start >= 0 and json_end > json_start:
            json_part = analysis_text[json_start:json_end]
            analysis = json.loads(json_part)
        else:
            # Fallback if JSON parsing fails
            raise ValueError("Could not extract valid JSON from the response")
        
    except Exception as e:
        print(f"[ERROR] Error analyzing message: {e}")
        # Fallback analysis if there's an error
        analysis = {
            "extracted_criteria": {},
            "action": "search_products" if "show me" in user_input.lower() or image is not None else "ask_for_criteria",
            "missing_criterion": "item_type",
            "question": "What type of clothing are you looking for?",
            "understanding": "I'm trying to understand what you're looking for.",
            "ready_to_search": False,
            "search_query": search_query
        }
    
    # Update session with extracted criteria - FIX THE ERROR HERE
    for criterion, value in analysis["extracted_criteria"].items():
        if value:  # Just check if value exists
            # Check type to prevent errors
            if isinstance(value, str) and value.lower() not in ("null", "none"):
                session["criteria_collected"][criterion] = value
            elif isinstance(value, (list, dict)):
                # Handle complex value types properly
                session["criteria_collected"][criterion] = value
            # Skip None values and empty strings
    
    # Handle different conversation actions
    if analysis["action"] == "greet":
        session["stage"] = "collecting"
        return [{"role": "assistant", "content": f"Hi! I'd be happy to help you find the perfect outfit. You can describe what you're looking for or upload an image. {analysis.get('question', 'What type of clothing are you looking for today?')}"}]
    
    elif analysis["action"] == "ask_for_criteria":
        session["stage"] = "collecting"
        question = analysis.get("question") or f"Can you tell me what kind of {analysis.get('missing_criterion', 'item')} you're looking for?"
        return [{"role": "assistant", "content": question}]
    
    elif analysis["action"] == "search_products" or analysis["action"] == "show_more":
        session["stage"] = "searching"
        # Use the existing retrieve_relevant_products and generate_rag_response functions
        search_query = analysis.get("search_query") or search_query
        
        # If we have image analysis, incorporate it into the search (if not already in search_query)
        if session.get("image_analysis") and session["image_analysis"].get("keywords"):
            # Check if keywords are already in the query
            keyword_present = False
            for keyword in session["image_analysis"]["keywords"]:
                if keyword.lower() in search_query.lower():
                    keyword_present = True
                    break
            
            if not keyword_present:
                keyword_str = " ".join(session["image_analysis"]["keywords"])
                search_query = f"{search_query} {keyword_str}"
                print(f"[DEBUG] Added image keywords to search query: {search_query}")
        
        # Get AI-generated recommendations
        print(f"[DEBUG] Calling generate_rag_response with query: {search_query}")
        try:
            recommendations = generate_rag_response(search_query, session["criteria_collected"], session)
            
            if not recommendations:
                # No products found - ask if user wants to broaden search
                session["recommendations_shown"] = True
                return [{"role": "assistant", "content": "I couldn't find any products matching your criteria. Could you be more general or try different options?"}]
            
            # Create structured response list
            response_list = []
            
            # First, summarize what we understood from their requirements
            criteria_display = {
                "item_type": "Item", 
                "style": "Style", 
                "color": "Color", 
                "occasion": "Occasion", 
                "price_range": "Price",
                "other": "Other"
            }
            
            criteria_summary = ", ".join([f"{criteria_display[k]}: {v}" for k, v in session["criteria_collected"].items() 
                                        if v and k in criteria_display])
            
            # If we have image analysis, mention it
            if session.get("image_analysis"):
                if criteria_summary:
                    response_list.append({"role": "assistant", "content": f"Based on your image and preferences ({criteria_summary}), here are some recommendations:"})
                else:
                    response_list.append({"role": "assistant", "content": f"Based on your image, here are some similar products:"})
            elif criteria_summary:
                response_list.append({"role": "assistant", "content": f"Here are some recommendations based on your preferences ({criteria_summary}):"})
            else:
                response_list.append({"role": "assistant", "content": "Here are some recommendations based on your request:"})
            
            # Add products with images and descriptions
            for rec in recommendations:
                try:
                    images = ast.literal_eval(rec["images"])
                except:
                    # Fallback if images can't be parsed
                    images = [rec["image"]]
                
                response_list.append({"role": "assistant", "content": rec["text"]})  # Product with AI explanation
                response_list.append({"role": "assistant", "content": gr.Gallery(images[:4], 
                                                                            columns=4, 
                                                                            rows=1, 
                                                                            object_fit="cover", 
                                                                            height="automatic",
                                                                            allow_preview=True)
                                                                            }) 
            
                response_list.append({"role": "assistant", "content": "\n\n\n\n\n\n"})  # Adds spacing
            
            # Add follow-up prompt
            response_list.append({"role": "assistant", "content": "Would you like to see more options, or should I help you find something else?"})
            
            # Update session
            session["recommendations_shown"] = True
            session["stage"] = "recommending"
            
            # Add assistant response to conversation history
            session["conversation_history"].append({"role": "assistant", "content": "I showed some product recommendations based on the user's criteria."})
            
            return response_list
        except Exception as e:
            print(f"[ERROR] Exception during recommendation generation: {e}")
            # Provide a fallback response
            return [{"role": "assistant", "content": "I'm sorry, I encountered an error while searching for products. Could you try a different query?"}]
    
    elif analysis["action"] == "restart":
        # Reset the session (redundant with the check at the beginning, but keeping for completeness)
        session["criteria_collected"] = {}
        session["stage"] = "collecting" 
        session["recommendations_shown"] = False
        session["conversation_history"] = []
        session["image_analysis"] = None
        return [{"role": "assistant", "content": "Let's start fresh! What are you looking for today? You can describe it or upload an image."}]
    
    elif analysis["action"] == "answer_question":
        # If the AI detects a question not related to product search
        if "question" in analysis and analysis["question"]:
            return [{"role": "assistant", "content": analysis["question"]}]
        else:
            return [{"role": "assistant", "content": "I'm here to help you find fashion products. What kind of item are you looking for?"}]
    
    # Default fallback response if analysis doesn't yield actionable results
    return [{"role": "assistant", "content": "I'm not sure I understood. Could you tell me what kind of clothing or fashion item you're looking for? You can also upload an image if you have one."}]

Step 7: 
- Gradio UI

In [234]:
# chat_ui = gr.ChatInterface(
#     fn=chat_fashion_assistant,
#     title="🛍️ ShopGPT",
#     description="Describe what you're looking for and get personalized recommendations!",
#     type="messages",  # Uses Gradio's structured chat format
#     theme='allenai/gradio-theme'
# )

# chat_ui.launch(share=True, server_port = 7877)


In [235]:
# # Create the chat interface with additional components
# with gr.Blocks(theme='allenai/gradio-theme') as app:
#     # Header section
#     with gr.Row():
#         with gr.Column(scale=8):
#             gr.Markdown("# 🛍️ ShopGPT")
#             gr.Markdown("Describe what you're looking for and get personalized recommendations!")
#         with gr.Column(scale=1):
#             close_button = gr.Button(
#                 "⏹️", # Unicode stop button symbol
#                 size="sm",
#                 variant="secondary",
#                 elem_classes="close-btn"
#             )
    
#     # Main chat interface
#     chatbot = gr.ChatInterface(
#         fn=chat_fashion_assistant,
#         type="messages",
#     )
    
#     # Add custom CSS
#     gr.HTML("""
#         <style>
#         .close-btn {
#             position: absolute;
#             top: 10px;
#             right: 10px;
#             min-width: 40px !important;
#             font-size: 12px;
#             opacity: 0.7;
#         }
#         .close-btn:hover {
#             opacity: 1;
#         }
#         </style>
#     """)
    
#     # Connect the button to the shutdown function
#     close_button.click(fn=lambda: app.close())

# # Launch with fixed port
# app.launch(share=True, server_port=7888)

In [236]:
# def create_fashion_interface():
#     """
#     Creates a reliable ShopGPT interface that properly displays responses
#     """
#     with gr.Blocks(theme='allenai/gradio-theme') as app:
#         gr.Markdown("# 🛍️ ShopGPT")
#         gr.Markdown("I'll help you find the perfect outfit! Describe what you're looking for or upload an image.")
        
#         # Simple chatbot with minimal configuration to avoid display issues
#         chatbot = gr.Chatbot(
#             height=600,
#             show_copy_button=False,
#             bubble_full_width=False,
#         )
        
#         # Add image preview area
#         with gr.Row(visible=False) as image_preview_row:
#             image_preview = gr.Image(
#                 type="filepath", 
#                 label="Uploaded Image", 
#                 height=150,
#                 interactive=False
#             )
        
#         with gr.Row():
#             # Text input
#             msg = gr.Textbox(
#                 placeholder="Describe what you're looking for or ask questions about fashion items...",
#                 label="Your message",
#                 show_label=False,
#                 scale=8
#             )
            
#             # Image upload button
#             image_btn = gr.UploadButton(
#                 "📷", 
#                 file_types=["image"], 
#                 file_count="single",
#                 scale=1
#             )
            
#             # Send button
#             submit_btn = gr.Button("Send", variant="primary", scale=1)
        
#         # Hidden state for storing the uploaded image
#         current_image = gr.State(None)
        
#         with gr.Row():
#             # Clear button
#             clear_btn = gr.Button("Start New Chat", scale=5)
            
#             # Discrete close server button
#             close_btn = gr.Button("Close Server", scale=1, variant="stop")
        
#         # Example queries
#         examples = gr.Examples(
#             examples=[
#                 ["I need a casual jacket for cool weather"],
#                 ["Looking for a formal dress for a wedding"],
#                 ["Show me some comfortable running shoes"],
#                 ["I need something professional for job interviews"]
#             ],
#             inputs=msg
#         )
        
#         # Update the current image when uploaded
#         def update_image(file):
#             if file:
#                 filename = file.name.split("/")[-1] if "/" in file.name else file.name
#                 return file.name, gr.Row(visible=True), file.name
#             return None, gr.Row(visible=False), None
        
#         # Update when image is uploaded
#         image_btn.upload(
#             update_image, 
#             image_btn, 
#             [current_image, image_preview_row, image_preview]
#         )
        
#         # Handle user message and generate response
#         def respond(message, image_path, chat_history):
#             # Skip if there's no input
#             if not message.strip() and not image_path:
#                 return chat_history, None, gr.Row(visible=False), None, ""
            
#             # First add user message to chat
#             if message.strip():
#                 chat_history.append([message, None])
#             elif image_path:
#                 chat_history.append(["[Looking for items similar to uploaded image]", None])
            
#             # Prepare image data if available
#             image_data = None
#             if image_path:
#                 try:
#                     from PIL import Image
#                     image_data = Image.open(image_path)
#                 except Exception as e:
#                     print(f"Error loading image: {e}")
            
#             # Get response from assistant
#             try:
#                 # Construct appropriate message
#                 query = message if message.strip() else "Find products like this image"
                
#                 # Call the assistant - ensure it returns a list of responses
#                 responses = chat_fashion_assistant(query, image_data, chat_history)
                
#                 # Make sure we have a valid response list
#                 if not responses or not isinstance(responses, list):
#                     # Fallback for empty or invalid response
#                     responses = [{"role": "assistant", "content": "I'm not sure how to respond to that. Could you try asking another way?"}]
                
#                 # Process each response
#                 for response in responses:
#                     # Handle each individual response
#                     if isinstance(response, dict) and "content" in response:
#                         # If the last message in chat_history has no assistant response yet, update it
#                         if chat_history and chat_history[-1][1] is None:
#                             chat_history[-1][1] = response["content"]
#                         else:
#                             # Otherwise add a new message pair with None for user and response for assistant
#                             chat_history.append([None, response["content"]])
#             except Exception as e:
#                 print(f"Error generating response: {e}")
#                 import traceback
#                 traceback.print_exc()
#                 # Add fallback response in case of exception
#                 if chat_history and chat_history[-1][1] is None:
#                     chat_history[-1][1] = "Sorry, I encountered an error. Please try again."
#                 else:
#                     chat_history.append([None, "Sorry, I encountered an error. Please try again."])
            
#             # Clear inputs after processing
#             return chat_history, None, gr.Row(visible=False), None, ""
        
#         # Define event handlers
#         submit_btn.click(
#             respond, 
#             inputs=[msg, current_image, chatbot], 
#             outputs=[chatbot, current_image, image_preview_row, image_preview, msg],
#             queue=True
#         )
        
#         msg.submit(
#             respond, 
#             inputs=[msg, current_image, chatbot], 
#             outputs=[chatbot, current_image, image_preview_row, image_preview, msg],
#             queue=True
#         )
        
#         # Clear everything 
#         def clear_all():
#             return [], None, gr.Row(visible=False), None, ""
        
#         clear_btn.click(clear_all, None, [chatbot, current_image, image_preview_row, image_preview, msg], queue=False)
        
#         # Add close server functionality with simple approach
#         def close_server():
#             app.close()
            
#         close_btn.click(close_server, None, None, queue=False)
        
#         # Add simple debug function to trace message flow
#         def trace_message_flow():
#             print("\nShopGPT message flow tracing is active")
#             print("Check console for error messages if responses don't appear\n")
#             return None
            
#         app.load(trace_message_flow, None, None)
        
#     return app

# # Create and launch the interface
# shop_gpt = create_fashion_interface()
# shop_gpt.launch(share=True, server_port = 7888)

In [237]:
def create_fashion_interface():
    """
    Creates a ShopGPT interface with actual image previews
    in both the input area and chat.
    """
    with gr.Blocks(theme='allenai/gradio-theme') as app:
        gr.Markdown("# 🛍️ ShopGPT")
        gr.Markdown("I'll help you find the perfect outfit! Describe what you're looking for or upload an image.")
        
        # Chatbot component
        chatbot = gr.Chatbot(
            height=600,
            show_copy_button=False,
            bubble_full_width=False,
        )
        
        # Loading indicator
        with gr.Row(visible=False) as loading_indicator:
            gr.Markdown("*Generating response...*")
        
        # Visual image preview above the textbox
        with gr.Row(visible=False) as image_preview_row:
            image_preview = gr.Image(
                label="Image Preview",
                show_label=False,
                height=150,
                interactive=False,
                type="filepath"  # Use filepath for compatibility
            )
        
        with gr.Row():
            # Text input
            msg = gr.Textbox(
                placeholder="Describe what you're looking for or ask questions about fashion items...",
                label="Your message",
                show_label=False,
                scale=8
            )
            
            # Image upload button
            image_btn = gr.UploadButton(
                "📷", 
                file_types=["image"], 
                file_count="single",
                scale=1
            )
            
            # Send button
            submit_btn = gr.Button("Send", variant="primary", scale=1)
        
        # Hidden state for storing the uploaded image
        current_image = gr.State(None)
        
        with gr.Row():
            # Clear button
            clear_btn = gr.Button("Start New Chat", scale=5)
            
            # Close server button
            close_btn = gr.Button("Close Server", scale=1, variant="stop")
        
        # Example queries
        examples = gr.Examples(
            examples=[
                ["I need a casual jacket for cool weather"],
                ["Looking for a formal dress for a wedding"],
                ["Show me some comfortable running shoes"],
                ["I need something professional for job interviews"]
            ],
            inputs=msg
        )
        
        # Show preview when image is uploaded
        def update_image_preview(file):
            if file and hasattr(file, 'name'):
                # Show the visual preview
                return file.name, gr.Row(visible=True), file.name
            return None, gr.Row(visible=False), None
        
        # Update when image is uploaded - show actual preview
        image_btn.upload(
            update_image_preview, 
            image_btn, 
            [current_image, image_preview_row, image_preview]
        )
        
        # STEP 1: Immediately show user message and image in chat
        def add_user_message(message, image_path, chat_history):
            # Skip if no input provided
            if not message.strip() and not image_path:
                return chat_history, message, gr.Button(interactive=True)
            
            # Add user message to chat
            if message.strip():
                chat_history.append([message, None])
            elif image_path:
                chat_history.append(["Looking for items similar to this image", None])
            
            # Add actual image to chat if present
            if image_path:
                try:
                    # Add image directly to chat history
                    chat_history.append([None, image_path])
                except Exception as e:
                    print(f"Error adding image to chat: {e}")
                    # Fallback to text if image display fails
                    chat_history.append([f"[Image uploaded]", None])
            
            # Clear text input and disable send button
            return chat_history, "", gr.Button(interactive=False)
        
        # STEP 2: Process the message and generate response
        def process_message(message, image_path, chat_history):
            # Skip if chat history is empty (edge case)
            if not chat_history:
                return chat_history, gr.Row(visible=False), None, gr.Button(interactive=True), gr.Row(visible=False)
            
            try:
                # Prepare image for processing (if available)
                image_data = None
                if image_path:
                    try:
                        from PIL import Image
                        image_data = Image.open(image_path)
                    except Exception as e:
                        print(f"Error loading image: {e}")
                
                # Construct query
                query = message if message.strip() else "Find products like this image"
                
                # Get response from assistant
                responses = chat_fashion_assistant(query, image_data, chat_history)
                
                # Process responses
                if responses and isinstance(responses, list):
                    for response in responses:
                        if isinstance(response, dict) and "content" in response:
                            chat_history.append([None, response["content"]])
                else:
                    # Fallback for invalid response
                    chat_history.append([None, "I'm not sure how to respond to that. Could you try asking another way?"])
            except Exception as e:
                print(f"Error generating response: {e}")
                import traceback
                traceback.print_exc()
                chat_history.append([None, "Sorry, I encountered an error processing your request. Please try again."])
            
            # Reset image preview, re-enable send button, and hide loading indicator
            return chat_history, gr.Row(visible=False), None, gr.Button(interactive=True), gr.Row(visible=False)
        
        # Split the response into two steps for immediate feedback
        submit_btn.click(
            fn=add_user_message,  # First immediately show user message
            inputs=[msg, current_image, chatbot],
            outputs=[chatbot, msg, submit_btn],
            queue=False  # No queue for immediate UI update
        ).then(
            fn=process_message,  # Then process the request (can take time)
            inputs=[msg, current_image, chatbot],
            outputs=[chatbot, image_preview_row, current_image, submit_btn, loading_indicator],
            queue=True  # Use queue for the actual processing
        )
        
        # Same pattern for text input submit
        msg.submit(
            fn=add_user_message,
            inputs=[msg, current_image, chatbot],
            outputs=[chatbot, msg, submit_btn],
            queue=False
        ).then(
            fn=process_message,
            inputs=[msg, current_image, chatbot],
            outputs=[chatbot, image_preview_row, current_image, submit_btn, loading_indicator],
            queue=True
        )
        
        # Clear chat and reset all states
        def clear_all():
            return [], None, "", gr.Row(visible=False), None, gr.Button(interactive=True)
        
        clear_btn.click(
            clear_all, 
            None, 
            [chatbot, current_image, msg, image_preview_row, image_preview, submit_btn], 
            queue=False
        )
        
        # Close server
        def close_server():
            app.close()
            
        close_btn.click(close_server, None, None, queue=False)
        
    return app

# Create and launch the interface
shop_gpt = create_fashion_interface()
shop_gpt.launch(share=True)

/var/folders/rj/sysxzl8x0q1dxp5rrjfyq19m0000gn/T/ipykernel_61537/834094478.py:11: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(
/var/folders/rj/sysxzl8x0q1dxp5rrjfyq19m0000gn/T/ipykernel_61537/834094478.py:11: DeprecationWarning: The 'bubble_full_width' parameter is deprecated and will be removed in a future version. This parameter no longer has any effect.
  chatbot = gr.Chatbot(


* Running on local URL:  http://127.0.0.1:7864


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


* Running on public URL: https://7e065bd01ff508c0f8.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


[ERROR] Error analyzing message: Expecting ',' delimiter: line 3 column 43 (char 72)
[DEBUG] Image detected in chat input
[DEBUG] Saved PIL image to: /var/folders/rj/sysxzl8x0q1dxp5rrjfyq19m0000gn/T/upload_2698b3f9.jpg
[DEBUG] Calling analyze_image_with_llava with path: /var/folders/rj/sysxzl8x0q1dxp5rrjfyq19m0000gn/T/upload_2698b3f9.jpg
[DEBUG] Starting image analysis for: /var/folders/rj/sysxzl8x0q1dxp5rrjfyq19m0000gn/T/upload_2698b3f9.jpg
[DEBUG] Query-aware analysis with: 'Find products like this image'
[DEBUG] LLaVA output will be saved to: /var/folders/rj/sysxzl8x0q1dxp5rrjfyq19m0000gn/T/llava_output.txt
[DEBUG] Executing LLaVA command


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


[DEBUG] LLaVA command completed successfully
[DEBUG] LLaVA output file size: 4258 characters
[DEBUG] Found marker line for description start
[DEBUG] Extracted description (544 chars): The most prominent clothing item in this image is a parka-style jacket. The jacket features a hood a...
[DEBUG] Found KEYWORDS section in output
[DEBUG] Successfully extracted 10 keywords: ['hooded', 'oversized', 'loose', 'fitting', 'parka', 'style', 'green', 'buttoned', 'detailing', 'drawstring']
[DEBUG] LLaVA analysis completed successfully ✅
[DEBUG] Image analysis complete. Description: The most prominent clothing item in this image is ...
[DEBUG] Extracted keywords: ['hooded', 'oversized', 'loose', 'fitting', 'parka', 'style', 'green', 'buttoned', 'detailing', 'drawstring']
[DEBUG] Enhanced search query: Find products like this image hooded oversized loose fitting parka style green buttoned detailing drawstring
[DEBUG] Calling generate_rag_response with query: green oversized hooded parka jacket
Searc